In [1]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import MIDILike, MIDITokenizer, REMI
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [3]:
attn_window = 10
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [4]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
directory = "/content/gdrive/MyDrive/ala/REMI/tokens_BPE"

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file                             

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

Streaming output truncated to the last 5000 lines.
batch number: 141545
input: [795, 37, 115, 134, 404, 6, 430, 340, 6, 430]
target: [37, 115, 134, 404, 6, 430, 340, 6, 430, 360]
-----
batch number: 141546
input: [6, 430, 18, 480, 25, 425, 30, 451, 37, 338]
target: [430, 18, 480, 25, 425, 30, 451, 37, 338, 45]
-----
batch number: 141547
input: [451, 331, 6, 430, 378, 6, 430, 18, 430, 25]
target: [331, 6, 430, 378, 6, 430, 18, 430, 25, 431]
-----
batch number: 141548
input: [30, 461, 37, 313, 325, 6, 430, 18, 111, 129]
target: [461, 37, 313, 325, 6, 430, 18, 111, 129, 25]
-----
batch number: 141549
input: [329, 23, 431, 35, 618, 50, 369, 404, 23, 431]
target: [23, 431, 35, 618, 50, 369, 404, 23, 431, 340]
-----
batch number: 141550
input: [23, 431, 360, 23, 431, 35, 618, 42, 370, 50]
target: [431, 360, 23, 431, 35, 618, 42, 370, 50, 369]
-----
batch number: 141551
input: [54, 305, 331, 23, 431, 378, 23, 431, 325, 23]
target: [305, 331, 23, 431, 378, 23, 431, 325, 23, 431]
-----
batch nu

In [6]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 142795
target_seq length: 142795


In [7]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [8]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 114236
train target: 114236
val input: 14280
val target: 14280
test input: 14279
test target: 14279


(None, None)

In [9]:
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

In [10]:
#creates tokenizer
tokenizer = REMI(params = '/content/gdrive/MyDrive/ala/REMI/tokens_noBPE/config.txt')

In [11]:
# Creates model
config = GPT2Config( 
    vocab_size=1000,
    n_positions=attn_window,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)                      

In [12]:
#model.from_pretrained("/content/gdrive/MyDrive/model_f/model2/")

In [13]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [14]:
num_epoch = 150

# Create a callback that saves the model's weights
cp_callback = [

# tf.keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/models_1/",
#                                                  verbose=1)

tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_pretrained("/content/gdrive/MyDrive/model_f/model5/"))
]

history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))


Epoch 1/150
3570/3570 [==============================] - 106s 23ms/step - loss: 6.1191 - accuracy: 0.0169 - val_loss: 5.5820 - val_accuracy: 0.0451
Epoch 2/150
3570/3570 [==============================] - 79s 22ms/step - loss: 5.2414 - accuracy: 0.0681 - val_loss: 4.8739 - val_accuracy: 0.0899
Epoch 3/150
3570/3570 [==============================] - 78s 22ms/step - loss: 4.7073 - accuracy: 0.1010 - val_loss: 4.4534 - val_accuracy: 0.1194
Epoch 4/150
3570/3570 [==============================] - 77s 22ms/step - loss: 4.3829 - accuracy: 0.1284 - val_loss: 4.1882 - val_accuracy: 0.1476
Epoch 5/150
3570/3570 [==============================] - 78s 22ms/step - loss: 4.1622 - accuracy: 0.1573 - val_loss: 3.9900 - val_accuracy: 0.1822
Epoch 6/150
3570/3570 [==============================] - 82s 23ms/step - loss: 3.9831 - accuracy: 0.1853 - val_loss: 3.8155 - val_accuracy: 0.2111
Epoch 7/150
3570/3570 [==============================] - 81s 23ms/step - loss: 3.8308 - accuracy: 0.2086 - val_loss: 

In [15]:
model.save_pretrained('/content/gdrive/MyDrive/model_f/REMI_model_1')

In [16]:
model.evaluate(x_test, y_test)

447/447 [==============================] - 3s 6ms/step - loss: 2.5311 - accuracy: 0.4128


[2.531132698059082, 0.41283002495765686]